In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
import json
import mrcfile
import umap

from cryo_sbi.inference.models import build_models
from cryo_sbi import CryoEmSimulator
from cryo_sbi.inference import priors
from cryo_sbi.utils.estimator_utils import sample_posterior, compute_latent_repr
from cryo_sbi.utils.image_utils import LowPassFilter, NormalizeIndividual, MRCtoTensor, FourierDownSample, Mask

In [ ]:
transform = transforms.Compose([
    MRCtoTensor(),
    transforms.Resize(size=(128, 128)),
    NormalizeIndividual(),
    Mask(128, 50)
])

transform_sim = transforms.Compose([
    NormalizeIndividual(),
    Mask(128, 45),
    LowPassFilter(128, 80)
])

transform_exp = transforms.Compose([
    MRCtoTensor(),
    transforms.Resize(size=(128, 128)),
    NormalizeIndividual(),
    #Mask(128, 45),   
    LowPassFilter(128, 80)
])

In [ ]:
file_name = '6wxb_nma'    # File name 
data_dir = "../experiments/6wxb/"
num_samples_stats = 20000           # Number of simulations for computing posterior stats
num_samples_SBC = 10000             # Number of simulations for SBC
num_posterior_samples_SBC = 4096    # Number of posterior samples for each SBC simulation
num_samples_posterior = 50000       # Number of samples to draw from posterior
batch_size_sampling = 100           # Batch size for sampling posterior
num_workers = 24                    # Number of CPU cores
device = 'cuda'                     # Device for computations
save_figures = False

## Load cryo-em simulator and posterior with correct config

In [ ]:
cryosbi = CryoEmSimulator(data_dir + "image_params_mixed_training.json")
cryosbi.config['SNR'] = 0.025
cryosbi.config['SIGMA'] = 6.0
cryosbi.config['DEFOCUS'] = 2.0

In [ ]:
train_config = json.load(open(data_dir + "resnet18_fft_encoder.json"))
estimator = build_models.build_npe_flow_model(train_config)
estimator.load_state_dict(torch.load(data_dir + "6wxb_mixed_posterior_noise.estimator"))
estimator.cuda()
estimator.eval();

# Test simulator over whole prior range

In [ ]:
indices = torch.tensor(np.arange(0, 100, 5, dtype=float))#torch.tensor(np.arange(0, cryosbi.max_index + 1, 5), dtype=float)
images = torch.stack([cryosbi.simulator(index) for index in indices], dim=0)

In [ ]:
masking = Mask(128, 50)

In [ ]:
images = masking(images)

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for idx, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[idx], vmax=4, vmin=-4, cmap='binary')
    ax.set_yticks([])
    ax.set_xticks([])
    ax.text(10, 20, str(int(indices[idx].item())))
#plt.savefig('../experiments/6wxb/results/plots/6wxb_examples.pdf')

In [ ]:
samples = sample_posterior(estimator, images, num_samples=10000, device='cuda')

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for idx, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(samples[:, idx].flatten().numpy(), bins=np.arange(0, 100, 1), histtype="step", color="blue", label="all")
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 100, 20))
    ax.axvline(indices[idx], color='red')
#plt.savefig('../experiments/6wxb/results/plots/torsion_nma.pdf', dpi=300)

# Simualte setup by generating particles from the original cryo-em structure

In [ ]:
indices = 50 * torch.ones(500, dtype=torch.float)
images = torch.stack([cryosbi.simulator(index) for index in indices], dim=0)

In [ ]:
samples_syn = sample_posterior(estimator, images, num_samples=10000, device='cuda')

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for idx, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(samples_syn[:, idx].flatten().numpy(), bins=np.arange(0, 100, 0.7), histtype="step", color="blue", label="all")
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 100, 20))
    ax.axvline(indices[idx], color='red')
#plt.savefig('../experiments/6wxb/results/plots/torsion_nma.pdf', dpi=300)

In [ ]:
plt.hist(samples_syn.mean(axis=0), bins=np.linspace(0, 100, 30))
plt.title('Experiment with synthetic data')
xticks = np.load('../../6wxb/6wxb_nma/distance_to_reference.npy')
plt.xlabel('Position of posterior means (RMSD)', fontsize=15)
plt.xticks(ticks=[0, 20, 40, 60, 80, 100], labels=list(map(lambda x: f'{x:.3f}', xticks[::20])))
plt.yticks([])

In [ ]:
xticks = np.load('../../6wxb/6wxb_nma/distance_to_reference.npy')
list(map(lambda x: f'{x:.3f}', xticks[::20]))

# Now with real data

In [ ]:
particles_transfomed = []
for i in range(1, 17):
    if i < 10:
        img_file = f'../../6wxb/particles/particles_0{i}.mrc'
    else:
        img_file = f'../../6wxb/particles/particles_{i}.mrc'
    tmp_images = transform(img_file)
    particles_transfomed.append(tmp_images)
particles_transfomed = torch.cat(particles_transfomed, dim=0)

In [ ]:
img_file = "../../6wxb/particles/bio_em_images.mrc"
particles_transfomed = transform(img_file)
#particles_transfomed[:, cm] = 0

In [ ]:
samples_real = sample_posterior(estimator, particles_transfomed, num_samples=10000, device='cuda', batch_size=500)

In [ ]:
offset = 14
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(particles_transfomed[i+offset], vmax=4, vmin=-4, cmap='binary')
    ax.set_yticks([])
    ax.set_xticks([])

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for i, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(samples_real[:, i+offset].flatten().numpy(), bins=np.arange(0, 100, 0.7), histtype="step", color="blue", label="all")
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 100, 20))
    ax.axvline(50, color='red')

In [ ]:
plt.title('Experiment with real data')
plt.hist(samples_real.mean(axis=0), bins=np.linspace(0, 100, 40))
xticks = np.load('../../6wxb/6wxb_nma/distance_to_reference.npy')
#plt.xlabel('Position of posterior means (RMSD)', fontsize=15)
#plt.xticks(ticks=[0, 20, 40, 60, 80, 100], labels=list(map(lambda x: f'{x:.3f}', xticks[::20])))
plt.yticks([])

In [ ]:
posterior_quantiles = np.quantile(samples_real.numpy(), [0.025, 0.975], axis=0)
confidence_widths = (posterior_quantiles[1] - posterior_quantiles[0]).flatten()
condition = confidence_widths < 50
idx = np.where(condition)[0]

In [ ]:
offset = 0
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(particles_transfomed[idx[i+offset]], vmax=4, vmin=-4, cmap='binary')
    ax.set_yticks([])
    ax.set_xticks([])

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for i, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(samples_real[:, idx[i+offset]].flatten().numpy(), bins=np.arange(0, 100, 1), histtype="step", color="blue", label="all")
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 100, 20))

In [ ]:
plt.hist(samples_real[:, idx].mean(axis=0), bins=np.linspace(0, 100, 30))
xticks = np.load('../../6wxb/6wxb_nma/distance_to_reference.npy')
#plt.xlabel('Position of posterior means (RMSD)', fontsize=15)
#plt.xticks(ticks=[0, 20, 40, 60, 80, 100], labels=list(map(lambda x: f'{x:.3f}', xticks[::20])))
plt.yticks([])

In [ ]:
particles_transfomed = []
for i in range(1, 17):
    if i < 10:
        img_file = f'../../6wxb/particles/particles_0{i}.mrc'
    else:
        img_file = f'../../6wxb/particles/particles_{i}.mrc'
    tmp_images = transform(img_file)
    particles_transfomed.append(tmp_images)
particles_transfomed = torch.cat(particles_transfomed, dim=0)

In [ ]:
cryosbi = CryoEmSimulator(data_dir + "image_params_mixed_training.json")

In [ ]:
num_syn_parts = 1000
#indices = 50 * torch.ones(num_syn_parts, dtype=torch.float)
indices = priors.get_uniform_prior_1d(cryosbi.max_index).sample((num_syn_parts,))
images = torch.stack([cryosbi.simulator(index) for index in indices], dim=0)
images = masking(images)

In [ ]:
plt.imshow(images[0])

In [ ]:
syntetic_particles_latent = compute_latent_repr(estimator, images, batch_size=100, device='cuda')
particles_latent = compute_latent_repr(estimator, particles_transfomed, batch_size=100, device='cuda')

In [ ]:
out_upper = (particles_latent < syntetic_particles_latent.max(dim=0).values).all(axis=1)
out_lower = (particles_latent > syntetic_particles_latent.min(dim=0).values).all(axis=1)
cond = torch.logical_and(out_lower, out_upper)
idx_almost_exp = torch.where(cond)[0]

In [ ]:
particles_latent = particles_latent[idx_almost_exp]

In [ ]:

cat_latent_samples = torch.cat((syntetic_particles_latent, particles_latent), dim=0)
labels_latent = torch.cat((torch.ones((len(indices),)), torch.zeros((particles_transfomed.shape[0],))), dim=0)

In [ ]:
reducer = umap.UMAP(metric='euclidean', n_components=2,  n_neighbors=1000)
embedding = reducer.fit_transform(cat_latent_samples.numpy())

In [ ]:
plt.scatter(embedding[:num_syn_parts, 0], embedding[:num_syn_parts, 1], s=3, c='blue', label='Synthetic images')
plt.scatter(embedding[num_syn_parts:, 0], embedding[num_syn_parts:, 1], s=3, c='red', label='Experimental images')
plt.xlabel('UMAP 1', fontsize=15)
plt.ylabel('UMAP 2', fontsize=15)
plt.legend(fontsize=15, markerscale=3, loc='lower left')

In [ ]:
#condition = np.sqrt(embedding[num_syn_parts:, 0] ** 2 + embedding[num_syn_parts:, 1] ** 2) < 5
#condition = embedding[num_syn_parts:, 0] > 10
condition = np.logical_and(embedding_exp[:, 0] > 7, embedding_exp[:, 0] < 8)
#condition = np.logical_and(embedding[:num_syn_parts, 0] < 10, embedding[:num_syn_parts, 1] > 12)
idx_almost_exp = np.where(condition)[0]
samples_almost = sample_posterior(estimator, particles_transfomed[idx_almost_exp], num_samples=10000, device='cuda')

In [ ]:
condition.sum()

In [ ]:
out_upper = (particles_latent < syntetic_particles_latent.max(dim=0).values).all(axis=1)
out_lower = (particles_latent > syntetic_particles_latent.min(dim=0).values).all(axis=1)
cond = torch.logical_and(out_lower, out_upper)
idx_almost_exp = torch.where(cond)[0]
#samples_almost = sample_posterior(estimator, particles_transfomed[idx_almost_exp], num_samples=10000, device='cuda')

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8), sharex=True)
for i, ax in enumerate(axes.reshape(1, -1)[0]):
    ax.hist(samples_almost[:, i].flatten().numpy(), bins=np.arange(0, 100, 2), histtype="step", color="blue", label="all")
    ax.set_yticks([])
    ax.set_yticks([])
    ax.set_xticks(range(0, 100, 20))
    ax.axvline(60, color='red')

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10, 8))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(particles_transfomed[idx_almost_exp[i]], vmax=4, vmin=-4, cmap='binary')
    ax.set_yticks([])
    ax.set_xticks([])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(particles_transfomed[1], vmax=3, vmin=-3, cmap='binary')
axes[0].set_title('Resized real particle')
axes[1].imshow(images[51], vmax=3, vmin=-3, cmap='binary')
axes[1].set_title('Syntehtic particle')
#fig.savefig('../experiments/6wxb/results/plots/comparison_particles.pdf', dpi=400)

In [ ]:
vis_images = transform_vis(images)
vis_images_real = transform_vis1('../../6wxb_test_particles/10532/data/03_Refined_Particles/P30_J363_particles/J342/localmotioncorrected/FoilHole_24136295_Data_24136362_24136364_20200224_020513_Fractions_particles_local_aligned.mrc')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(vis_images_real[43], vmax=3, vmin=-3, cmap='binary')
axes[0].set_title('Resized real particle')
axes[1].imshow(vis_images[51], vmax=3, vmin=-3, cmap='binary')
axes[1].set_title('Syntehtic particle')

In [ ]:
 torch.fft.fft2(images[8])

In [ ]:
def circular_mask(n_pixels, radius, inside=True):
    grid = torch.linspace(-0.5 * (n_pixels - 1), 0.5 * (n_pixels - 1), n_pixels)
    r_2d = grid[None, :] ** 2 + grid[:, None] ** 2

    if inside is True:
        mask = r_2d < radius**2
    else:
        mask = r_2d > radius**2

    return mask

In [ ]:
import torch.distributions as d

class AddLowFrequencyNoise:
    """Adding low frequency noise, also serving as a low pass filter"""
    def __init__(self, image_size, min_frequency, amplitude) -> None:
        self._imge_size = image_size
        self._mask = circular_mask(image_size, min_frequency, inside=False)
        self._num_frequencies = self._mask.sum()
        
        if isinstance(amplitude, (float, int)):
            self._amplitude = d.uniform.Uniform(low=amplitude, high=amplitude)
        elif isinstance(amplitude, (list, tuple)):
            self._amplitude = d.uniform.Uniform(low=amplitude[0], high=amplitude[1])
    
    def __call__(self, image):
        fft_image = torch.fft.fft2(image)

        if len(image.shape) == 2:
            fft_image[self._mask] += self._amplitude.sample() * torch.randn((self._num_frequencies,), dtype=torch.complex64)
            fft_image[self._mask == False] = 0 + 0j
        elif len(image.shape) == 3:
            fft_image[:, self._mask] += self._amplitude.sample((image.shape[0], 1)) * torch.randn((image.shape[0], self._num_frequencies), dtype=torch.complex64)
            fft_image[:, self._mask == False] = 0 + 0j
        else:
            raise NotImplementedError
    
        return torch.fft.ifft2(fft_image).real


In [ ]:
d.uniform.Uniform(low=torch.tensor(1., device='cuda'), high=torch.tensor(1., device='cuda')).sample((4,))

In [ ]:
cm = circular_mask(128, 80, inside=False)
cm2 = circular_mask(128, 50, inside=False)

In [ ]:
idx = 10

In [ ]:
fft_img = torch.fft.fft2(images[idx])
print(fft_img.real.max())
noise = (fft_img.real.std() * 3.0 * torch.randn((cm.sum(),), dtype=torch.complex64))
#fft_img[cm] += noise
fft_img[cm2] = 0 + 0j

In [ ]:
ni = torch.fft.ifft2(fft_img).real

In [ ]:
images[:, cm2] = 0

In [ ]:
t = AddLowFrequencyNoise(128, 80, (0, 500))
images = t(images)

In [ ]:
torch.tensor([1,]).device = 

In [ ]:
plt.imshow(images[43], cmap='binary', vmax=2, vmin=-2)

In [ ]:
plt.imshow(ni, cmap='binary', vmax=2, vmin=-2)

In [ ]:
plt.imshow(particles_transfomed[43], cmap='binary', vmax=2, vmin=-2)

In [ ]:
cryosbi = CryoEmSimulator(data_dir + "image_params_mixed_training.json")
cryosbi.config['SNR'] = 0.1
cryosbi.config['SIGMA'] = 15.0
cryosbi.config['DEFOCUS'] = 1.0

In [ ]:
def gen_img(coord, image_params):
    n_atoms = coord.shape[1]

    if isinstance(image_params["SIGMA"], float):
        atom_sigma = image_params["SIGMA"]

    elif isinstance(image_params["SIGMA"], list) and len(image_params["SIGMA"]) == 2:
        atom_sigma = np.random.uniform(
            low=image_params["SIGMA"][0], high=image_params["SIGMA"][1]
        )
    else:
        raise ValueError(
            "SIGMA should be a single value or a list of [min_sigma, max_sigma]"
        )

    norm = 1 / (2 * torch.pi * atom_sigma**2 * n_atoms)

    grid_min = -image_params["PIXEL_SIZE"] * (image_params["N_PIXELS"] - 1) * 0.5
    grid_max = (
        image_params["PIXEL_SIZE"] * (image_params["N_PIXELS"] - 1) * 0.5
        + image_params["PIXEL_SIZE"]
    )

    grid = torch.arange(grid_min, grid_max, image_params["PIXEL_SIZE"])

    gauss_x = torch.exp(-0.5 * (((grid[:, None] - coord[0, :]) / atom_sigma) ** 2))

    gauss_y = torch.exp(-0.5 * (((grid[:, None] - coord[1, :]) / atom_sigma) ** 2))

    image = torch.matmul(gauss_x, gauss_y.T) * norm

    return image


In [ ]:
gen_img(cryosbi.models[0], cryosbi.config)

In [ ]:
%timeit gen_img(cryosbi.models[0], cryosbi.config)

In [ ]:
%timeit gen_img(cryosbi.models[0], cryosbi.config)